In [8]:
import os
import pandas as pd
import json
import os, sys
print(os.path.join(os.path.dirname(os.getcwd()), "app"))

sys.path.append("/Users/officialbiznas/Documents/GitHub/stockFries/app")
import scraper
import config
import requests
from bs4 import BeautifulSoup

/Users/officialbiznas/Documents/GitHub/stockFries/app


## Testing Each Function

In [4]:
def forms_scraper(fund_link):
    """Takes the SEC fund link to return the 2 most recent filing URLS"""
    r = requests.get(fund_link)
    soup = BeautifulSoup(r.content, 'html.parser')
    filing_links = []
    table = soup.find_all('table')[2]
    rows = [i for i in table.find_all('tr')]
    for i in rows:
        cols = i.find('td')
        if cols is not None:
            if '13F-HR' in cols.text:
                base_link = 'https://www.sec.gov'
                link = i.find('a').get('href')
                filing_links.append(base_link + link)

    form_links = []
    form_dict = {}
    for i in filing_links[:2]:
        r = requests.get(i)
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('table')
        rows = [i for i in table.find_all('tr')]
        form_links.append(base_link + rows[3].find('a').get('href'))
        form_dict[]
    return form_links


def shares_scraper(form_link):
    """Takes a filing link and appends all holdings to a list"""
    r = requests.get(form_link)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all('table')[3]
    rows = table.find_all('tr')[3:]

    shares_list = []
    rows_text = []
    for i in rows:
        cols = i.find_all('td')
        cols_text = []
        for col in cols:
            if col is not None:
                cols_text.append(col.text)
        rows_text.append(cols_text)

    for i in rows_text:
        if (i[6] != 'Put') or (i[6] != 'Call'):
            shares_list.append({'CUSIP': i[2][:9], 'Company': i[0], 'Value': int(i[3].replace(',', '')),
                                'Shares': int(i[4].replace(',', ''))})

    return shares_list
def pandas_analysis_test(fund_dict):
    """Takes the fund dict and gets the filings URLS from forms_scraper, then extends two aggregate lists for the current and previous holdings across all funds before analysis in pandas and export to json txt"""
    list_current = []
    list_previous = []
    for k, v in fund_dict.items():
        try:
            forms = forms_scraper(v)
            print(forms)
            shares_list_current = shares_scraper(forms[0])
            list_current.extend(shares_list_current)
            shares_list_previous = shares_scraper(forms[1])
            list_previous.extend(shares_list_previous)
        except:
            pass
    print(list_current)
    df_current = pd.DataFrame(list_current)
    dfc = df_current.groupby('CUSIP').agg(
        {'Company': 'first', 'Value': 'sum', 'Shares': 'sum'})
    dfc['Company'] = dfc['Company'].str.title()
    dfc['Ownership'] = (dfc['Value'] / dfc['Value'].sum()) * 100
    dfc = dfc.sort_values('Ownership', ascending=False).round(2)

    df_previous = pd.DataFrame(list_previous)
    dfp = df_previous.groupby('CUSIP').agg(
        {'Company': 'first', 'Value': 'sum', 'Shares': 'sum'})
    dfp['Company'] = dfp['Company'].str.title()

    dff = pd.merge(dfc, dfp, on=['CUSIP', 'Company'], how='outer')
    dff['Change'] = ((dff['Shares_x'] - dff['Shares_y']) /
                     dff['Shares_x']) * 100

    json_df = dff.round(2).to_json(orient='split')
    #with open('os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt'), 'w') as f:
    #    json.dump(json_df, f)
    return dff

In [5]:
fund_link_test = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1535392&owner=exclude&count=40&hidefilings=0'
form_links_test = forms_scraper(fund_link_test)

In [9]:
config.fund_dict

{'Mangrove Partners': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1535392&owner=exclude&count=40&hidefilings=0',
 'Segantii': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1544676&owner=exclude&count=40&hidefilings=0',
 'MIG Capital': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1425649&owner=exclude&count=40&hidefilings=0',
 'Parametrica Asset Management Ltd': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1535090&owner=exclude&count=40&hidefilings=0',
 'Masters Capital Management': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1104186&owner=exclude&count=40&hidefilings=0',
 'Harvest Capital Strategies': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1226355&owner=exclude&count=40&hidefilings=0',
 'Anson Funds': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1491072&owner=exclude&count=40&hidefilings=0',
 'Capital Fund Management': 'https://www.sec.gov/cgi-bin/

In [6]:
shares_list_test = shares_scraper(form_links_test[0])
print(shares_list_test)

[{'CUSIP': '03076C106', 'Company': 'AMERIPRISE FINL INC', 'Value': 6497, 'Shares': 43301}, {'CUSIP': '04878Q863', 'Company': 'ATLANTIC POWER CORP', 'Value': 4405, 'Shares': 2202685}, {'CUSIP': 'G0772R208', 'Company': 'BANK OF NT BUTTERFIELD&SON L', 'Value': 1508, 'Shares': 61828}, {'CUSIP': '07987C204', 'Company': 'BELLUS HEALTH INC NEW', 'Value': 2138, 'Shares': 207778}, {'CUSIP': '09609G100', 'Company': 'BLUEBIRD BIO INC', 'Value': 7912, 'Shares': 129622}, {'CUSIP': '097793400', 'Company': 'BONANZA CREEK ENERGY INC', 'Value': 30540, 'Shares': 2060700}, {'CUSIP': '10948W103', 'Company': 'BRIGHTSPHERE INVT GROUP INC', 'Value': 28124, 'Shares': 2257173}, {'CUSIP': '110122157', 'Company': 'BRISTOL-MYERS SQUIBB CO', 'Value': 14127, 'Shares': 3946145}, {'CUSIP': '12477X106', 'Company': 'CAI INTERNATIONAL INC', 'Value': 14086, 'Shares': 845512}, {'CUSIP': '14888U101', 'Company': 'CATALYST PHARMACEUTICALS INC', 'Value': 14462, 'Shares': 3130195}, {'CUSIP': '16383L106', 'Company': 'CHEMOCENTR

In [7]:
%%time
df_test = pandas_analysis_test(config.fund_dict)

['https://www.sec.gov/Archives/edgar/data/1535392/000156761920015183/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1535392/000156761920010282/xslForm13F_X01/form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1544676/000154467620000008/xslForm13F_X01/13f.xml', 'https://www.sec.gov/Archives/edgar/data/1544676/000154467620000006/xslForm13F_X01/13fv2.xml']
['https://www.sec.gov/Archives/edgar/data/1425649/000110465920095369/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1425649/000110465920062428/xslForm13F_X01/infotable.xml']
[]
['https://www.sec.gov/Archives/edgar/data/1104186/000110418620000005/xslForm13F_X01/t13f-0620.xml', 'https://www.sec.gov/Archives/edgar/data/1104186/000110418620000004/xslForm13F_X01/t13f-0320.xml']
['https://www.sec.gov/Archives/edgar/data/1226355/000122635519000012/xslForm13F_X01/Harvest13f_3q2019.inftab.xml', 'https://www.sec.gov/Archives/edgar/data/1226355/000122635519000009/xslForm13F_X01

['https://www.sec.gov/Archives/edgar/data/919468/000117266119001064/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/919468/000117266119000232/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1517137/000092189520002245/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1517137/000092189520001444/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1647251/000164725120000006/xslForm13F_X01/Form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1647251/000164725120000004/xslForm13F_X01/Form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1167483/000091957420005337/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1167483/000091957420003646/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1345471/000134547120000054/xslForm13F_X01/infochart13F0630amend2.xml', 'https://www.sec.gov/Archives/edgar/data/1345471/000134547120000044/xslForm13F_

In [7]:
df_test.tail()

,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change
CUSIP,,,,,,,
N6596X109,Nxp Semiconductors Nv,NaN,NaN,NaN,2128.0,25663.0,NaN
N70544106,Playa Hotels & Resorts Nv,NaN,NaN,NaN,4103.0,2344296.0,NaN
Y2573F102,Flex Ltd,NaN,NaN,NaN,98359.0,11744352.0,NaN
Y41053102,International Seaways Inc,NaN,NaN,NaN,21923.0,917646.0,NaN
Y75638109,Atlas Corp,NaN,NaN,NaN,1423.0,185009.0,NaN


## Tetsting SRC Code

In [2]:
%%time
scraper.pandas_analysis(fund_dict)

['https://www.sec.gov/Archives/edgar/data/1535392/000156761920015183/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1535392/000156761920010282/xslForm13F_X01/form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1544676/000154467620000008/xslForm13F_X01/13f.xml', 'https://www.sec.gov/Archives/edgar/data/1544676/000154467620000006/xslForm13F_X01/13fv2.xml']
['https://www.sec.gov/Archives/edgar/data/1425649/000110465920095369/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1425649/000110465920062428/xslForm13F_X01/infotable.xml']
[]
['https://www.sec.gov/Archives/edgar/data/1104186/000110418620000005/xslForm13F_X01/t13f-0620.xml', 'https://www.sec.gov/Archives/edgar/data/1104186/000110418620000004/xslForm13F_X01/t13f-0320.xml']
['https://www.sec.gov/Archives/edgar/data/1226355/000122635519000012/xslForm13F_X01/Harvest13f_3q2019.inftab.xml', 'https://www.sec.gov/Archives/edgar/data/1226355/000122635519000009/xslForm13F_X01

['https://www.sec.gov/Archives/edgar/data/919468/000117266119001064/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/919468/000117266119000232/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1517137/000092189520002245/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1517137/000092189520001444/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1647251/000164725120000006/xslForm13F_X01/Form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1647251/000164725120000004/xslForm13F_X01/Form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1167483/000091957420005337/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1167483/000091957420003646/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1345471/000134547120000054/xslForm13F_X01/infochart13F0630amend2.xml', 'https://www.sec.gov/Archives/edgar/data/1345471/000134547120000044/xslForm13F_

In [3]:
fn = os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt') #app/json_df.txt
print(fn)
with open(fn, 'r') as f:
        json_df = json.load(f)
df = pd.read_json(json_df, orient='split').reset_index()
df.tail()

/Users/officialbiznas/Documents/GitHub/stockFries/app/json_df.txt


,index,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change
2612,N6596X109,Nxp Semiconductors Nv,NaN,NaN,NaN,2128.0,25663.0,NaN
2613,N70544106,Playa Hotels & Resorts Nv,NaN,NaN,NaN,4103.0,2344296.0,NaN
2614,Y2573F102,Flex Ltd,NaN,NaN,NaN,98359.0,11744352.0,NaN
2615,Y41053102,International Seaways Inc,NaN,NaN,NaN,21923.0,917646.0,NaN
2616,Y75638109,Atlas Corp,NaN,NaN,NaN,1423.0,185009.0,NaN


In [2]:
fn = os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt') #app/json_df.txt
print(fn)
with open(fn, 'r') as f:
        json_df = json.load(f)
df = pd.read_json(json_df, orient='split').reset_index()
df.tail()

/Users/officialbiznas/Documents/GitHub/stockFries/app/json_df.txt


,index,Company,Shares_x,Value_x,Ownership,Shares_y,Value_y,Change
2256,N70544,Playa Hotels & Resorts Nv,NaN,NaN,NaN,2344296.0,4103.0,NaN
2257,N72482,Qiagen Nv,NaN,NaN,NaN,54200000.0,63232.0,NaN
2258,Y2573F,Flex Ltd,NaN,NaN,NaN,11744352.0,98359.0,NaN
2259,Y41053,International Seaways Inc,NaN,NaN,NaN,917646.0,21923.0,NaN
2260,Y75638,Atlas Corp,NaN,NaN,NaN,185009.0,1423.0,NaN


In [17]:
df.head()

,index,Company,Shares_x,Value_x,Ownership,Shares_y,Value_y,Change
0,88160R,Tesla Inc,42116277.0,192177977.0,2.94,NaN,NaN,NaN
1,097023,Boeing Co,3658828.0,167087119.0,2.56,394102.0,58777.0,89.23
2,64110L,Netflix Inc,5576767.0,151505143.0,2.32,9129848.0,3428260.0,-63.71
3,037833,Apple Inc,375380.0,136938624.0,2.09,347481.0,88361.0,7.43
4,30231G,Exxon Mobil Corp,3060000.0,136843200.0,2.09,3707932.0,140056.0,-21.17


In [13]:
stringer = '''<!DOCTYPE html>
<html>
<body>

<tr><table class="tableFile2" summary="Results">
         <tr>
            <th width="7%" scope="col">Filings</th>
            <th width="10%" scope="col">Format</th>
            <th scope="col">Description</th>
            <th width="10%" scope="col">Filing Date</th>
            <th width="15%" scope="col">File/Film Number</th>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761920015183/0001567619-20-015183-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-20-015183&nbsp;(34 Act)&nbsp; Size: 28 KB            </td>
            <td>2020-08-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>201101730         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000090514820000681/0000905148-20-000681-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000905148-20-000681 Size: 127 KB            </td>
            <td>2020-06-10</td>
         <td></td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761920010282/0001567619-20-010282-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-20-010282&nbsp;(34 Act)&nbsp; Size: 37 KB            </td>
            <td>2020-05-15</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>20881491         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189520001308/0000921895-20-001308-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-20-001308 Size: 297 KB            </td>
            <td>2020-05-07</td>
         <td></td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189520000860/0000921895-20-000860-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-20-000860 Size: 299 KB            </td>
            <td>2020-03-23</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761920003664/0001567619-20-003664-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-20-003664&nbsp;(34 Act)&nbsp; Size: 38 KB            </td>
            <td>2020-02-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>20616054         </td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189520000192/0000921895-20-000192-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-20-000192 Size: 303 KB            </td>
            <td>2020-01-29</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13G</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000090514820000074/0000905148-20-000074-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Statement of acquisition of beneficial ownership by individuals<br />Acc-no: 0000905148-20-000074 Size: 133 KB            </td>
            <td>2020-01-13</td>
         <td></td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13G</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000090514820000073/0000905148-20-000073-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Statement of acquisition of beneficial ownership by individuals<br />Acc-no: 0000905148-20-000073 Size: 139 KB            </td>
            <td>2020-01-13</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761919021334/0001567619-19-021334-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-19-021334&nbsp;(34 Act)&nbsp; Size: 51 KB            </td>
            <td>2019-11-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>191217236         </td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761919016768/0001567619-19-016768-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-19-016768&nbsp;(34 Act)&nbsp; Size: 45 KB            </td>
            <td>2019-08-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>191024110         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761919011215/0001567619-19-011215-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-19-011215&nbsp;(34 Act)&nbsp; Size: 37 KB            </td>
            <td>2019-05-15</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>19828391         </td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189519000747/0000921895-19-000747-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-19-000747 Size: 304 KB            </td>
            <td>2019-03-14</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761919004097/0001567619-19-004097-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-19-004097&nbsp;(34 Act)&nbsp; Size: 29 KB            </td>
            <td>2019-02-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>19602956         </td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189518003392/0000921895-18-003392-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-18-003392 Size: 320 KB            </td>
            <td>2018-12-27</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189518003071/0000921895-18-003071-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-18-003071 Size: 360 KB            </td>
            <td>2018-11-15</td>
         <td></td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761918006104/0001567619-18-006104-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-18-006104&nbsp;(34 Act)&nbsp; Size: 24 KB            </td>
            <td>2018-11-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>181182114         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189518002715/0000921895-18-002715-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-18-002715 Size: 249 KB            </td>
            <td>2018-10-09</td>
         <td></td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761918000829/0001567619-18-000829-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001567619-18-000829&nbsp;(34 Act)&nbsp; Size: 18 KB            </td>
            <td>2018-08-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>181015370         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036118024013/0001140361-18-024013-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-18-024013&nbsp;(34 Act)&nbsp; Size: 18 KB            </td>
            <td>2018-05-15</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>18837501         </td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189518001368/0000921895-18-001368-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-18-001368 Size: 239 KB            </td>
            <td>2018-04-17</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036118008108/0001140361-18-008108-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-18-008108&nbsp;(34 Act)&nbsp; Size: 19 KB            </td>
            <td>2018-02-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>18611722         </td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189518000138/0000921895-18-000138-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-18-000138 Size: 271 KB            </td>
            <td>2018-01-12</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036117042845/0001140361-17-042845-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-17-042845&nbsp;(34 Act)&nbsp; Size: 16 KB            </td>
            <td>2017-11-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>171202367         </td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036117031852/0001140361-17-031852-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-17-031852&nbsp;(34 Act)&nbsp; Size: 14 KB            </td>
            <td>2017-08-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>171029388         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189517001966/0000921895-17-001966-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-17-001966 Size: 196 KB            </td>
            <td>2017-07-21</td>
         <td></td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036117020342/0001140361-17-020342-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-17-020342&nbsp;(34 Act)&nbsp; Size: 12 KB            </td>
            <td>2017-05-15</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>17842287         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036117007093/0001140361-17-007093-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-17-007093&nbsp;(34 Act)&nbsp; Size: 14 KB            </td>
            <td>2017-02-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>17607510         </td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189517000076/0000921895-17-000076-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-17-000076 Size: 208 KB            </td>
            <td>2017-01-23</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036116086074/0001140361-16-086074-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-16-086074&nbsp;(34 Act)&nbsp; Size: 15 KB            </td>
            <td>2016-11-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>161992072         </td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189516005539/0000921895-16-005539-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-16-005539 Size: 235 KB            </td>
            <td>2016-08-18</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036116076453/0001140361-16-076453-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-16-076453&nbsp;(34 Act)&nbsp; Size: 16 KB            </td>
            <td>2016-08-15</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>161831087         </td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036116065553/0001140361-16-065553-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-16-065553&nbsp;(34 Act)&nbsp; Size: 20 KB            </td>
            <td>2016-05-16</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>161652881         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189516004022/0000921895-16-004022-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-16-004022 Size: 228 KB            </td>
            <td>2016-04-07</td>
         <td></td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036116052861/0001140361-16-052861-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-16-052861&nbsp;(34 Act)&nbsp; Size: 20 KB            </td>
            <td>2016-02-16</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>161425688         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189516003126/0000921895-16-003126-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-16-003126 Size: 288 KB            </td>
            <td>2016-02-01</td>
         <td></td>
         </tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036115041401/0001140361-15-041401-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-15-041401&nbsp;(34 Act)&nbsp; Size: 21 KB            </td>
            <td>2015-11-16</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>151232511         </td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000114036115031974/0001140361-15-031974-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Quarterly report filed by institutional managers, Holdings<br />Acc-no: 0001140361-15-031974&nbsp;(34 Act)&nbsp; Size: 20 KB            </td>
            <td>2015-08-14</td>
            <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br>151053761         </td>
         </tr>
<tr>
<td nowrap="nowrap">SC 13G</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189515001456/0000921895-15-001456-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" >Statement of acquisition of beneficial ownership by individuals<br />Acc-no: 0000921895-15-001456 Size: 301 KB            </td>
            <td>2015-05-22</td>
         <td></td>
         </tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000092189515001453/0000921895-15-001453-index.htm" id="documentsbutton">&nbsp;Documents</a></td>
<td class="small" ><b>[Amend]</b> General statement of acquisition of beneficial ownership<br />Acc-no: 0000921895-15-001453 Size: 332 KB            </td>
            <td>2015-05-21</td>
         <td></td>
         </tr>
      </table>

</body>
</html>
'''

In [15]:
#r = requests.get(fund_link)
soup = BeautifulSoup(stringer, 'html.parser')

In [16]:
soup

<!DOCTYPE html>

<html>
<body>
<tr><table class="tableFile2" summary="Results">
<tr>
<th scope="col" width="7%">Filings</th>
<th scope="col" width="10%">Format</th>
<th scope="col">Description</th>
<th scope="col" width="10%">Filing Date</th>
<th scope="col" width="15%">File/Film Number</th>
</tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000156761920015183/0001567619-20-015183-index.htm" id="documentsbutton"> Documents</a></td>
<td class="small">Quarterly report filed by institutional managers, Holdings<br/>Acc-no: 0001567619-20-015183 (34 Act)  Size: 28 KB            </td>
<td>2020-08-14</td>
<td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-15348&amp;owner=exclude&amp;count=40">028-15348</a><br/>201101730         </td>
</tr>
<tr class="blueRow">
<td nowrap="nowrap">SC 13D/A</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1535392/000090514820000681/0000905148-20-000681-index.htm" id="docume

In [54]:
import re
filing_links = []
filing_dates = []
table = soup.find_all('table')[0]
rows = [i for i in table.find_all('tr')]
for i in rows:
    cols = i.find('td')
    if cols is not None:
        if '13F-HR' in cols.text:
            base_link = 'https://www.sec.gov'
            link = i.find('a').get('href')
            #print('iiiiii')
            #print(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})\n",(i.text)).group())
            #print(i.text)
            ##rint(i.find_next('td'))
            #print(i)
            #print(link)
            filing_links.append(base_link + link)
            date = re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})\n",(i.text)).group()
            print('date:::::', date)
            filing_dates.append(date)
max_date = max(filing_dates)
print(max_date)

date::::: 2020-08-14

date::::: 2020-05-15

date::::: 2020-02-14

date::::: 2019-11-14

date::::: 2019-08-14

date::::: 2019-05-15

date::::: 2019-02-14

date::::: 2018-11-14

date::::: 2018-08-14

date::::: 2018-05-15

date::::: 2018-02-14

date::::: 2017-11-14

date::::: 2017-08-14

date::::: 2017-05-15

date::::: 2017-02-14

date::::: 2016-11-14

date::::: 2016-08-15

date::::: 2016-05-16

date::::: 2016-02-16

date::::: 2015-11-16

date::::: 2015-08-14

2020-08-14



In [ ]:
%%time
df_test = pandas_analysis_test(config.fund_dict)